In [162]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

# importing machine learning models for prediction
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC

#PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale 
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error
from sklearn import model_selection

pd.options.display.max_columns = 999

In [163]:
# Read data
train_values = pd.read_csv('data/train_values.csv')
train_labels = pd.read_csv('data/train_labels.csv')
test_values = pd.read_csv('data/test_values.csv')

In [164]:
# PCA
X = train_values.iloc[:,15:25]
X = pd.get_dummies(X, prefix_sep='_')

pca = PCA()
principalComponents = pca.fit_transform(scale(X))

PCA_df = pd.DataFrame(data=principalComponents[:,:5], columns=['PC1','PC2','PC3','PC4','PC5'])

# Add PCA components to DF
new_train_values = pd.concat([train_values, PCA_df], axis=1)

# Drop columns used for PCA
# columns #15 ~ #25: "count_floors_pre_eq" ~ "plan_configuration"
drop_list = new_train_values.columns[15:26]
new_train_values.drop(drop_list, axis=1, inplace=True)

In [165]:
new_train_df = pd.merge(new_train_values, train_labels, on="building_id")

In [234]:
geo1_count = pd.DataFrame(new_train_df['geo_level_1_id'].value_counts().sort_index())
geo2_count = pd.DataFrame(new_train_df['geo_level_2_id'].value_counts().sort_index())
geo3_count = pd.DataFrame(new_train_df['geo_level_3_id'].value_counts().sort_index())

geo1_count.rename(columns={"geo_level_1_id":"counts"}, inplace=True)
geo2_count.rename(columns={"geo_level_2_id":"counts"}, inplace=True)
geo3_count.rename(columns={"geo_level_3_id":"counts"}, inplace=True)

In [235]:
index_geo1 = list(range(31))
a = pd.DataFrame()
a['id'] = index_geo1
geo1_count=pd.merge(a, geo1_count, how='left', left_on='id', right_index=True)
geo1_count=geo1_count.interpolate()

In [236]:
dam1 = []

for i in range(0,31):
    dam1_count = len(new_train_df[new_train_df.geo_level_1_id==i][new_train_df.damage_grade==1])
    dam1.append(dam1_count)

geo1_count["damage_1"] = pd.DataFrame(dam1)

dam2 = []

for i in range(0,31):
    dam2_count = len(new_train_df[new_train_df.geo_level_1_id==i][new_train_df.damage_grade==2])
    dam2.append(dam2_count)

geo1_count["damage_2"] = pd.DataFrame(dam2)

dam3 = []

for i in range(0,31):
    dam3_count = len(new_train_df[new_train_df.geo_level_1_id==i][new_train_df.damage_grade==3])
    dam3.append(dam3_count)

geo1_count["damage_3"] = pd.DataFrame(dam3)

geo1_count['geo1_dam1_prob'] = geo1_count['damage_1'] / geo1_count['counts']
geo1_count['geo1_dam2_prob'] = geo1_count['damage_2'] / geo1_count['counts']
geo1_count['geo1_dam3_prob'] = geo1_count['damage_3'] / geo1_count['counts']

/var/folders/gm/kys_b_lx5n153wpkc5l01s280000gn/T/ipykernel_49724/1945352171.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dam1_count = len(new_train_df[new_train_df.geo_level_1_id==i][new_train_df.damage_grade==1])
/var/folders/gm/kys_b_lx5n153wpkc5l01s280000gn/T/ipykernel_49724/1945352171.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dam2_count = len(new_train_df[new_train_df.geo_level_1_id==i][new_train_df.damage_grade==2])
/var/folders/gm/kys_b_lx5n153wpkc5l01s280000gn/T/ipykernel_49724/1945352171.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dam3_count = len(new_train_df[new_train_df.geo_level_1_id==i][new_train_df.damage_grade==3])


In [238]:
index_geo2 = list(range(1428))
a = pd.DataFrame()
a['id'] = index_geo2
geo2_count=pd.merge(a, geo2_count, how='left', left_on='id', right_index=True)
geo2_count=geo2_count.interpolate()
#geo2_count.drop("id", axis=1, inplace=True)


In [239]:
dam1 = []

for i in range(0,1428):
    dam1_count = len(new_train_df[new_train_df.geo_level_2_id==i][new_train_df.damage_grade==1])
    dam1.append(dam1_count)

geo2_count["damage_1"] = pd.DataFrame(dam1)

dam2 = []

for i in range(0,1428):
    dam2_count = len(new_train_df[new_train_df.geo_level_2_id==i][new_train_df.damage_grade==2])
    dam2.append(dam2_count)

geo2_count["damage_2"] = pd.DataFrame(dam2)

dam3 = []

for i in range(0,1428):
    dam3_count = len(new_train_df[new_train_df.geo_level_2_id==i][new_train_df.damage_grade==3])
    dam3.append(dam3_count)

geo2_count["damage_3"] = pd.DataFrame(dam3)
geo2_count=geo2_count.interpolate()

geo2_count['geo2_dam1_prob'] = geo2_count['damage_1'] / geo2_count['counts']
geo2_count['geo2_dam2_prob'] = geo2_count['damage_2'] / geo2_count['counts']
geo2_count['geo2_dam3_prob'] = geo2_count['damage_3'] / geo2_count['counts']

/var/folders/gm/kys_b_lx5n153wpkc5l01s280000gn/T/ipykernel_49724/3585502810.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dam1_count = len(new_train_df[new_train_df.geo_level_2_id==i][new_train_df.damage_grade==1])
/var/folders/gm/kys_b_lx5n153wpkc5l01s280000gn/T/ipykernel_49724/3585502810.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dam2_count = len(new_train_df[new_train_df.geo_level_2_id==i][new_train_df.damage_grade==2])
/var/folders/gm/kys_b_lx5n153wpkc5l01s280000gn/T/ipykernel_49724/3585502810.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dam3_count = len(new_train_df[new_train_df.geo_level_2_id==i][new_train_df.damage_grade==3])


In [241]:
index_geo3 = list(range(12568))
b = pd.DataFrame()
b['id'] = index_geo3
geo3_count=pd.merge(b, geo3_count, how='left', left_on='id', right_index=True)
geo3_count=geo3_count.interpolate()
##geo3_count.drop("id", axis=1, inplace=True)

In [242]:
dam1 = []

for i in range(0,12568):
    dam1_count = len(new_train_df[new_train_df.geo_level_3_id==i][new_train_df.damage_grade==1])
    dam1.append(dam1_count)

geo3_count["damage_1"] = pd.DataFrame(dam1)

dam2 = []

for i in range(0,12568):
    dam2_count = len(new_train_df[new_train_df.geo_level_3_id==i][new_train_df.damage_grade==2])
    dam2.append(dam2_count)

geo3_count["damage_2"] = pd.DataFrame(dam2)

dam3 = []

for i in range(0,12568):
    dam3_count = len(new_train_df[new_train_df.geo_level_3_id==i][new_train_df.damage_grade==3])
    dam3.append(dam3_count)

geo3_count["damage_3"] = pd.DataFrame(dam3)
geo3_count=geo3_count.interpolate()

geo3_count['geo3_dam1_prob'] = geo3_count['damage_1'] / geo3_count['counts']
geo3_count['geo3_dam2_prob'] = geo3_count['damage_2'] / geo3_count['counts']
geo3_count['geo3_dam3_prob'] = geo3_count['damage_3'] / geo3_count['counts']

/var/folders/gm/kys_b_lx5n153wpkc5l01s280000gn/T/ipykernel_49724/3103804941.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dam1_count = len(new_train_df[new_train_df.geo_level_3_id==i][new_train_df.damage_grade==1])
/var/folders/gm/kys_b_lx5n153wpkc5l01s280000gn/T/ipykernel_49724/3103804941.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dam2_count = len(new_train_df[new_train_df.geo_level_3_id==i][new_train_df.damage_grade==2])
/var/folders/gm/kys_b_lx5n153wpkc5l01s280000gn/T/ipykernel_49724/3103804941.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dam3_count = len(new_train_df[new_train_df.geo_level_3_id==i][new_train_df.damage_grade==3])


In [269]:
prob_table = new_train_df.iloc[:,0:4]


,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id
0,802906,6,487,12198
1,28830,8,900,2812
2,94947,21,363,8973
3,590882,22,418,10694
4,201944,11,131,1488
...,...,...,...,...
260596,688636,25,1335,1621
260597,669485,17,715,2060
260598,602512,17,51,8163
260599,151409,26,39,1851


In [270]:
temp = geo1_count.iloc[:,[0,5,6,7]]
prob_table = pd.merge(prob_table, temp, how="left", left_on="geo_level_1_id", right_on="id")
prob_table.drop("id", axis=1, inplace=True)

temp2 = geo2_count.iloc[:,[0,5,6,7]]
prob_table = pd.merge(prob_table, temp2, how="left", left_on="geo_level_2_id", right_on="id")
prob_table.drop("id", axis=1, inplace=True)

temp3 = geo3_count.iloc[:,[0,5,6,7]]
prob_table = pd.merge(prob_table, temp3, how="left", left_on="geo_level_3_id", right_on="id")
prob_table.drop("id", axis=1, inplace=True)

prob_table.drop(["geo_level_1_id", "geo_level_2_id", "geo_level_3_id"], axis=1, inplace=True)

In [274]:
new_train_df = pd.merge(new_train_df, prob_table, how="left", left_on="building_id", right_on="building_id")

In [277]:
drop_list2 = new_train_df.columns[14:28]
new_train_df.drop(drop_list2, axis=1, inplace=True)

In [279]:
new_train_df = pd.get_dummies(new_train_df, prefix_sep='_')

In [280]:
new_train_labels = new_train_df['damage_grade']
new_train_df.drop(['building_id','damage_grade','geo_level_1_id','geo_level_2_id','geo_level_3_id'], axis=1, inplace=True)

In [282]:
############################
### Clean Test Values
############################

In [283]:
# PCA
X2 = test_values.iloc[:,15:25]
X2 = pd.get_dummies(X2, prefix_sep='_')

pca_test = PCA()
principalComponents_test = pca_test.fit_transform(scale(X2))

PCA_df_test = pd.DataFrame(data=principalComponents_test[:,:5], columns=['PC1','PC2','PC3','PC4','PC5'])

test_values = pd.concat([test_values, PCA_df_test], axis=1)

# Drop columns used for PCA
# columns #15 ~ #25: "count_floors_pre_eq" ~ "plan_configuration"
drop_list2 = test_values.columns[15:26]
test_values.drop(drop_list2, axis=1, inplace=True)

In [285]:
prob_table2 = test_values.iloc[:,0:4]

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id
0,300051,17,596,11307
1,99355,6,141,11987
2,890251,22,19,10044
3,745817,26,39,633
4,421793,17,289,7970
...,...,...,...,...
86863,310028,4,605,3623
86864,663567,10,1407,11907
86865,1049160,22,1136,7712
86866,442785,6,1041,912


In [286]:
temp = geo1_count.iloc[:,[0,5,6,7]]
prob_table2 = pd.merge(prob_table2, temp, how="left", left_on="geo_level_1_id", right_on="id")
prob_table2.drop("id", axis=1, inplace=True)

temp2 = geo2_count.iloc[:,[0,5,6,7]]
prob_table2 = pd.merge(prob_table2, temp2, how="left", left_on="geo_level_2_id", right_on="id")
prob_table2.drop("id", axis=1, inplace=True)

temp3 = geo3_count.iloc[:,[0,5,6,7]]
prob_table2 = pd.merge(prob_table2, temp3, how="left", left_on="geo_level_3_id", right_on="id")
prob_table2.drop("id", axis=1, inplace=True)

prob_table2.drop(["geo_level_1_id", "geo_level_2_id", "geo_level_3_id"], axis=1, inplace=True)

In [287]:
test_values = pd.merge(test_values, prob_table2, how="left", left_on="building_id", right_on="building_id")

In [290]:
drop_list3 = test_values.columns[14:28]
test_values.drop(drop_list3, axis=1, inplace=True)

In [292]:
test_values = pd.get_dummies(test_values, prefix_sep='_')
test_values.drop(['building_id','geo_level_1_id','geo_level_2_id','geo_level_3_id'], axis=1, inplace=True)

In [296]:
new_train_df.to_csv('V3_train_feat_engineered.csv')
new_train_labels.to_csv('V3_train_labels_feat_engineered.csv')
test_values.to_csv('V3_test_feat_engineered.csv')